# Graph matrix subspace checking

In [1]:
#import os
#os.environ["MKL_NUM_THREADS"] = "4" 
#os.environ["NUMEXPR_NUM_THREADS"] = "4" 
#os.environ["OMP_NUM_THREADS"] = "4" 

### 1. First results

In [1]:
from tqdm import tqdm 
import numpy as np
import scipy as sp
import os
import datetime
from pprint import pprint

from graph_utils import *

In [77]:
def get_rank_and_nullspace(A, get_cutoff_dif=False, verbose=False):
    
    _, v, d = sp.linalg.svd(A, lapack_driver="gesdd")
        
    # такой выбор толеранса используется в матлабе и в нампае
    # https://numpy.org/doc/stable/reference/generated/numpy.linalg.matrix_rank.html
    tol = v.max() * max(A.shape) * np.finfo(A.dtype).eps
    nnz = (v >= tol).sum()
    
    if verbose:
        print("eigen vals:", d)
        print("cutoff:", d[nnz])
    
    if get_cutoff_dif == False:
        return nnz, d[nnz:].T

In [43]:
def a_is_subspace_of_b(basis_a, basis_b, get_cutoff=False):
    
    orig_rank, _ = get_rank_and_nullspace(basis_b)
    combined_rank, _ = get_rank_and_nullspace(np.hstack([basis_a, basis_b]))
    
    return orig_rank == combined_rank

In [44]:
def find_chain_starts(basis_set):
    
    # Sort by the dimensionality of subspaces
    basis_set = sorted(basis_set, key=lambda x: x[1])
    chain_starts = [basis_set[0]]
    
    # Go over basis sets and check whether their spaces are subspaces of some of chain_starts spaces
    for i in range(1, len(basis_set)):
        
        not_a_subspace = True
        
        for start in chain_starts:
            if a_is_subspace_of_b(basis_set[i][2], start[2]):
                not_a_subspace = False
                break
        
        if not_a_subspace:
            chain_starts.append(basis_set[i])
    
    return chain_starts

In [45]:
# Function for an online version of an algorithm
# chain_starts are supposed to be sorted by x[1]

def get_new_starts(chain_starts, new_basis):
    
    if len(chain_starts) == 0:
        chain_starts.append(new_basis)
        return
    
    checked_pos = 0
    
    # check if we are embedded in some space
    while checked_pos < len(chain_starts) and len(chain_starts[checked_pos][2][0]) >= len(new_basis[2][0]):
        if a_is_subspace_of_b(new_basis[2], chain_starts[checked_pos][2]):
            return
        checked_pos += 1
    
    # new basis is not embedded anywhere therefore we insert it
    chain_starts.insert(checked_pos, new_basis)
    checked_pos += 1
    
    # now we delete subspaces, that are embedded into a new one
    while checked_pos < len(chain_starts):
        if a_is_subspace_of_b(chain_starts[checked_pos][2], new_basis[2]):
            chain_starts.pop(checked_pos)
        else:
            checked_pos += 1

In [11]:
ALL_GRAPHS = get_graphs()
del ALL_GRAPHS['cycle']

# ALL_GRAPHS = {x: ALL_GRAPHS[x] for x in ['center', 'sinking_ship']}
# что-то в center баг какой-то 'sinking_ship'

In [ ]:
%%time

NON_EQUIV = True

for name, g in ALL_GRAPHS.items():
    
    print(name)
    I = find_stable_sets(g)
    A, var_matr, num_params = build_triple_equalities(I, g.shape[0])
    
    num_combs = total_num_of_combinations(I, var_matr, num_params, only_nonequivalent=NON_EQUIV)
    
    chain_starts = []
    
    for i, (addA, addB, opening) in tqdm(enumerate(build_variance_equalities_iterator(
            I, var_matr, num_params, only_nonequivalent=NON_EQUIV)), total=num_combs):
        rank, null_basis = get_rank_and_nullspace(np.array(A + addA, dtype=np.float64))
        if len(null_basis) != 0:
            get_new_starts(chain_starts, [i, rank, null_basis, opening])
    
    print("{} x {}".format(len(A), len(A[0])), "{} x {}".format(len(addA), len(addA[0])))
    print(len(chain_starts), "dimensions:", ",".join([str(len(x[2][0])) for x in chain_starts]))
    
    print("openings:")
    for x in chain_starts: 
        pprint(x[-1])

### 2. Speed optimizations

In [7]:
ALL_GRAPHS = {'dupl': ALL_GRAPHS['dupl'], 'zigzag': ALL_GRAPHS['zigzag'], 'fork': ALL_GRAPHS['fork'], 
              'big_triag': ALL_GRAPHS['big_triag'], 'big_zig': ALL_GRAPHS['big_zig']}

In [8]:
# storm has multiple bases
# ship stil doesn't cut it.. we'll have to try more cores

# if you simply switch lapack driver to gesvd - results don't change but it takes twice longer

In [ ]:
%%time

for name, g in ALL_GRAPHS.items():

    print(name)
    I = find_stable_sets(g)
    A, var_matr, num_params = build_triple_equalities(I, g.shape[0])

    basis_set = []
    for i, addA in enumerate(build_variance_equalities_iterator(I, var_matr, num_params, only_nonequivalent=True)):
        basis_set.append([i, *get_rank_and_nullspace(np.array(A + addA[0], dtype=np.float64))])

    chain_starts = find_chain_starts(basis_set)

    print("{} x {}".format(len(A), len(A[0])), "{} x {}".format(len(addA[0]), len(addA[0][0])))
    print(len(chain_starts), "dimensions:", ",".join([str(len(x[2][0])) for x in chain_starts]))

In [9]:
%%time
# it has been 2min29sec
# with 8 cores it's 2min58sec
# with 4 cores it's 3min3sec

for name, g in ALL_GRAPHS.items():
    
    print(name)
    I = find_stable_sets(g)
    A, var_matr, num_params = build_triple_equalities(I, g.shape[0])
    
    chain_starts = []
    for i, addA in enumerate(build_variance_equalities_iterator(I, var_matr, num_params, only_nonequivalent=True)):
        chain_starts.append([i, *get_rank_and_nullspace(np.array(A + addA[0], dtype=np.float64))])
        chain_starts = find_chain_starts(chain_starts)
    
    print("{} x {}".format(len(A), len(A[0])), "{} x {}".format(len(addA[0]), len(addA[0][0])))
    print(len(chain_starts), "dimensions:", ",".join([str(len(x[2][0])) for x in chain_starts]))

storm
19 x 45 35 x 45
2 dimensions: 7,7
CPU times: user 12min 5s, sys: 9.58 s, total: 12min 15s
Wall time: 3min 3s


In [12]:
%%time

maxdif = 1e-19

for name, g in ALL_GRAPHS.items():
    
    print(name)
    I = find_stable_sets(g)
    A, var_matr, num_params = build_triple_equalities(I, g.shape[0])
    
    chain_starts = []
    for i, addA in enumerate(build_variance_equalities_iterator(I, var_matr, num_params, only_nonequivalent=True)):
        
        matrix = np.array(A + addA[0], dtype=np.float64)
        _, v, d = sp.linalg.svd(matrix, lapack_driver="gesdd")
        tol = v.max() * max(matrix.shape) * np.finfo(matrix.dtype).eps
        nnz = (v >= tol).sum()
        
        if nnz < len(v):
            maxdif = max(v[nnz] / v[nnz-1], maxdif)
            
        get_new_starts(chain_starts, [i, nnz, d[nnz:].T])
    
    print("{} x {}".format(len(A), len(A[0])), "{} x {}".format(len(addA[0]), len(addA[0][0])))
    print(len(chain_starts), "dimensions:", ",".join([str(len(x[2][0])) for x in chain_starts]))

storm
19 x 45 35 x 45
2 dimensions: 7,7
CPU times: user 17min 15s, sys: 16.3 s, total: 17min 32s
Wall time: 2min 11s


#### no aplas optimizations

mkl is used, 4 cores

online (stupid): 1min 32s
offline: 1min 18s
online (smarter): 1min 25s 

#### limitting number of cores

#os.environ["MKL_NUM_THREADS"] = "1" 
#os.environ["NUMEXPR_NUM_THREADS"] = "1" 
#os.environ["OMP_NUM_THREADS"] = "1" 

online (stupid): 1min 33s
offline: 1min 20s
online (smarter): 1min 26s

#### mkl on 8 cores - same result

**so online version isn't much slower and number of cores doesn't affect the results much**

### 3. Storm analysis

In [27]:
g = ALL_GRAPHS['storm']
I = find_stable_sets(g)
A, var_matr, num_params = build_triple_equalities(I, g.shape[0])

А он правильно задан?

In [28]:
is_copirr(g)

True

Судя по обсуждениям в интернете - могут быть проблемы при применении gesdd на плохо обсуловленных матрицах - поэтому все эксперименты будем пробовать и с gesdd и gesvd 

Смотрим на максимальное соотношение между первым отрезанным и неотрезанным с.ч. и минимальное между неотрезанными с.ч. в определении подпространств 

In [60]:
max_cut_dif  = 1e-19
max_real_dif = 1e-19
min_rank = 1e3

total_num = total_num_of_combinations(I, var_matr, num_params, only_nonequivalent=True)

for addA, addB, opening in tqdm(
    build_variance_equalities_iterator(I, var_matr, num_params, only_nonequivalent=True), total=total_num
):

    matrix = np.array(A + addA, dtype=np.float64)
    _, v, d = sp.linalg.svd(matrix, lapack_driver="gesdd")
    tol = v.max() * max(matrix.shape) * np.finfo(matrix.dtype).eps
    nnz = (v >= tol).sum()
        
    if nnz < len(v):
        max_cut_dif = max(v[nnz] / v[nnz-1], max_cut_dif)
        max_real_dif = max(v[0] / v[nnz-1], max_real_dif)
        min_rank = min(nnz, min_rank)
        
print(max_cut_dif, max_real_dif)

100%|██████████| 131072/131072 [01:27<00:00, 1493.97it/s]

1.619397193511745e-14 36.66232538729725


In [62]:
len(A[0])-min_rank

7

In [59]:
max_cut_dif  = 1e-19
max_real_dif = 1e-19

total_num = total_num_of_combinations(I, var_matr, num_params, only_nonequivalent=True)

for addA, addB, opening in tqdm(
    build_variance_equalities_iterator(I, var_matr, num_params, only_nonequivalent=True), total=total_num
):

    matrix = np.array(A + addA, dtype=np.float64)
    _, v, d = sp.linalg.svd(matrix, lapack_driver="gesvd")
    tol = v.max() * max(matrix.shape) * np.finfo(matrix.dtype).eps
    nnz = (v >= tol).sum()
        
    if nnz < len(v):
        max_cut_dif = max(v[nnz] / v[nnz-1], max_cut_dif)
        max_real_dif = max(v[0] / v[nnz-1], max_real_dif)
            
print(max_cut_dif, max_real_dif)


100%|██████████| 131072/131072 [04:23<00:00, 497.56it/s]

5.682405378060707e-15 36.662325387297344


Проверим, найдётся ли среди отфильтрованных нами пространств 8-мерное

In [49]:
min_rank = 1e3

total_num = total_num_of_combinations(I, var_matr, num_params, only_nonequivalent=False)

for addA, addB, opening in tqdm(
    build_variance_equalities_iterator(I, var_matr, num_params, only_nonequivalent=False), total=total_num
):

    matrix = np.array(A + addA, dtype=np.float64)
    v = sp.linalg.svd(matrix, lapack_driver="gesdd", compute_uv=False)
    tol = v.max() * max(matrix.shape) * np.finfo(matrix.dtype).eps
    nnz = (v >= tol).sum()
    min_rank = min(nnz, min_rank)

  0%|          | 83446/34359738368 [00:35<4046:52:39, 2358.45it/s]

KeyboardInterrupt: 

Ну, этого нам не проверить

#### Проверим на коположительность матрицы из этих подпространств и из их объединения

In [12]:
ALL_GRAPHS = get_graphs()

In [13]:
import scipy.io

#### 1. Сначала попробуем для с7

In [18]:
g = ALL_GRAPHS['cycle']
I = find_stable_sets(g)
A, var_matr, num_params = build_triple_equalities(I, g.shape[0])

In [14]:
def get_center_point(g, var_matr):
    
    I = find_stable_sets(g)
    
    point = np.ones(int(len(g) * (len(g)-1) / 2)) * 1/3
    
    for x in range(len(I)):
        for y in range(x+1, len(I)):
            if len(I[x].intersection(I[y])) == 2:
                inter = I[x].intersection(I[y])
                i = I[x].difference(I[y]).pop()
                j = inter.pop()
                k = inter.pop()
                l = I[y].difference(I[x]).pop()

                point[var_matr[i,l]] = 1
    
    return point

In [15]:
def get_matrix_at_point(var_matr, point):
    
    matrix = np.ones(var_matr.shape)
    
    for i in range(0, var_matr.shape[0]):
        for j in range(i+1, var_matr.shape[0]):
            matrix[i,j] = matrix[j,i] = -np.cos(np.pi * point[var_matr[i, j]]) 
    
    return matrix

In [16]:
def check_lem6(var_matr, I, point):
    
    matrix = get_matrix_at_point(var_matr, point)
    min_val = 0
    
    for (i, j, k) in I:
        u = [
            np.sin(np.pi * point[var_matr[j,k]]), 
            np.sin(np.pi * point[var_matr[i,k]]), 
            np.sin(np.pi * point[var_matr[i,j]])
        ]
        for row in matrix:
            new_val = row[i]*u[0] + row[j]*u[1] + row[k]*u[2]
            if new_val < min_val:
                min_val = new_val
                
    return min_val

In [19]:
center = get_center_point(g, var_matr)
get_matrix_at_point(var_matr, center)

array([[ 1. , -0.5, -0.5,  1. ,  1. , -0.5, -0.5],
       [-0.5,  1. , -0.5, -0.5,  1. ,  1. , -0.5],
       [-0.5, -0.5,  1. , -0.5, -0.5,  1. ,  1. ],
       [ 1. , -0.5, -0.5,  1. , -0.5, -0.5,  1. ],
       [ 1. ,  1. , -0.5, -0.5,  1. , -0.5, -0.5],
       [-0.5,  1. ,  1. , -0.5, -0.5,  1. , -0.5],
       [-0.5, -0.5,  1. ,  1. , -0.5, -0.5,  1. ]])

In [20]:
check_lem6(var_matr, I, center)

-2.220446049250313e-16

Это меньше машинного эпсилона, так что всё выполнено.

Загоняем матрицу в матлабовсий код -> получаем коположительность.  
Теперь пройдёмся по окрестности, для этого сгенерируем случайное направление, лежащее в подпространстве, удовлетворяющем леммам 3 и 4 (генерируем в нём случайную матрицу и вычитаем из центральной) и прибавляем его к центральной матрице с каким-то небольшим коэффициентом.  

Новую матрицу надо выбрать так, чтобы хотя бы лемма 6 выполнялась.

In [64]:
 for i, (addA, addB, opening) in enumerate(build_variance_equalities_iterator(
            I, var_matr, num_params, only_nonequivalent=True)):
        rank, null_basis = get_rank_and_nullspace(np.array(A + addA, dtype=np.float64))
        break

In [68]:
np.array(A + addA) 

ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 54 is different from 1)

In [25]:
totalA = np.array(A + addA, dtype=np.float64)

In [21]:
def get_random_direction(basis):
    coords = np.random.randn(basis.shape[1])
    r_point = basis @ coords 
    #return (r_point - center) / np.linalg.norm(r_point - center)
    return r_point / np.linalg.norm(r_point)

In [27]:
max_val = -1e10
step = 1e-4

_, basis = get_rank_and_nullspace(totalA)

direction = get_random_direction(basis)

while check_lem6(var_matr, I, center + step * direction) < -np.finfo(np.float64).eps:
    direction = get_random_direction(basis)
    if check_lem6(var_matr, I, center + step * direction) > max_val:
        max_val = check_lem6(var_matr, I, center + step * direction)
        print(max_val)

-4.440892098500626e-16
-2.7755575615628914e-16
-2.220446049250313e-16


#### Означает ли это, что мы лежим супер близко с границей Au > 0?

In [31]:
direction = get_random_direction(basis)
point = center + 0.1 * direction
s=get_matrix_at_point(var_matr, point)

In [32]:
s

array([[ 1.        , -0.43320848, -0.4482434 ,  0.99697857,  0.99808212,
        -0.47039633, -0.52266758],
       [-0.43320848,  1.        , -0.61149404, -0.50190949,  0.99984917,
         0.99913109, -0.54196185],
       [-0.4482434 , -0.61149404,  1.        , -0.37745257, -0.59765978,
         0.99605138,  0.99637458],
       [ 0.99697857, -0.50190949, -0.37745257,  1.        , -0.51685531,
        -0.45817377,  0.99848018],
       [ 0.99808212,  0.99984917, -0.59765978, -0.51685531,  1.        ,
        -0.52412156, -0.46888985],
       [-0.47039633,  0.99913109,  0.99605138, -0.45817377, -0.52412156,
         1.        , -0.50646447],
       [-0.52266758, -0.54196185,  0.99637458,  0.99848018, -0.46888985,
        -0.50646447,  1.        ]])

In [33]:
check_lem6(var_matr, I, point)

-2.7755575615628914e-16

* На одной и той же матрице код выдавал разные результаты - добавил толеранс в сравнении с нулём. Хотя бы с центральной точкой сработало.
* Думал, что проблема в округлении в матлабе - там просто обрезалось число при принте. Надо было прописать format long g
* Может выбрать направление получше?

In [34]:
scipy.io.savemat('a', {'data':s}, do_compression=False)

Для многих направлений даже с небольшим шагом не получается.
* Попробовать для упрощённого вида, который мы получали руками
* Проверить, что для матриц, которые мы получаем при шаге хотя бы выполняются условия лемм 3, 4 и 6 (шаг точно нужно выбирать такой, чтобы 6 лемма выполнялась)

* Получилось для с7 - не получалось из-за того, что я направление неправильно задавал ((r_point - center) / np.linalg.norm(r_point - center))
* Лемма 6 сломалась из-за того, что я не отшкалировал углы обратно при подсчёте сертификата

#### 2. с7 но явно решённый и с доп условиями

In [133]:
from numpy import cos, array

def get_simplified_matrix(p):
    
    return np.array([
        [1, -cos(p[0]), cos(p[0]+p[1]), -cos(p[0]+p[1]+p[2]), -cos(p[4]+p[5]+p[6]), cos(p[5]+p[6]), -cos(p[6])],
        [-cos(p[0]), 1, -cos(p[1]), cos(p[1]+p[2]), -cos(p[1]+p[2]+p[3]), -cos(p[5]+p[6]+p[0]), cos(p[6]+p[0])],
        [cos(p[0]+p[1]), -cos(p[1]), 1, -cos(p[2]), cos(p[2]+p[3]), -cos(p[2]+p[3]+p[4]), -cos(p[0]+p[1]+p[6])],
        [-cos(p[0]+p[1]+p[2]), cos(p[1]+p[2]), -cos(p[2]), 1, -cos(p[3]), cos(p[3]+p[4]), -cos(p[3]+p[4]+p[5])],
        [-cos(p[4]+p[5]+p[6]), -cos(p[1]+p[2]+p[3]), cos(p[2]+p[3]), -cos(p[3]), 1, -cos(p[4]), cos(p[4]+p[5])],
        [cos(p[5]+p[6]), -cos(p[5]+p[6]+p[0]), -cos(p[2]+p[3]+p[4]), cos(p[3]+p[4]), -cos(p[4]), 1, -cos(p[5])],
        [-cos(p[6]), cos(p[6]+p[0]),  -cos(p[0]+p[1]+p[6]), -cos(p[3]+p[4]+p[5]), cos(p[4]+p[5]), -cos(p[5]), 1]
    ])

In [129]:
center = np.ones(7) * np.pi / 3

In [132]:
get_simplified_matrix(center)

array([[ 1. , -0.5, -0.5,  1. ,  1. , -0.5, -0.5],
       [-0.5,  1. , -0.5, -0.5,  1. ,  1. , -0.5],
       [-0.5, -0.5,  1. , -0.5, -0.5,  1. ,  1. ],
       [ 1. , -0.5, -0.5,  1. , -0.5, -0.5,  1. ],
       [ 1. ,  1. , -0.5, -0.5,  1. , -0.5, -0.5],
       [-0.5,  1. ,  1. , -0.5, -0.5,  1. , -0.5],
       [-0.5, -0.5,  1. ,  1. , -0.5, -0.5,  1. ]])

Код говорит, что она коположительна.

**Берём код для генерирования углов из этого множества, который мы использовали раньше**

In [134]:
arrays = [array([1.8132337 , 0.7133784 , 1.24228641, 1.08119275, 2.06039991,
       1.08119275, 1.22802568]),
array([0.09622127, 2.92404257, 0.15690484, 2.12285946, 0.64200562,
       0.16953895, 1.34689091]),
array([0.10894777, 0.66488034, 2.16113198, 0.42573434, 0.22430197,
       0.72638459, 1.58305346]),
array([1.00000000e-15, 2.47912546e+00, 6.62467189e-01, 2.47912546e+00,
       6.53451004e-01, 1.17165633e-01, 3.02442702e+00]),
array([0.00622218, 1.55895093, 0.65362154, 0.17717865, 0.40227136,
       0.41735264, 1.91960064]),
array([0.35438032, 0.68575076, 2.4558419 , 0.68575076, 0.96654673,
       1.86193204, 1.23382903]),
array([1.00000000e-15, 7.59130735e-01, 1.13263579e+00, 1.10054730e-01,
       8.99968818e-01, 4.90795672e-01, 2.65079698e+00]),
array([1.00676967, 1.41438199, 0.46689436, 1.37652027, 1.76507238,
       1.37652027, 1.01195225]),
array([1.35397235, 1.21730731, 0.570313  , 2.40851029, 0.73308236,
       0.66199742, 2.47959523]),
array([1.65396582, 0.8478992 , 2.10177085, 0.93435163, 0.14933715,
       1.71377453, 1.42781813])]

In [ ]:
s = get_simplified_matrix(arrays[9])

scipy.io.savemat('a', {'data':s}, do_compression=False)

Для 4, 6, 7, 8 это коположительный матрицы, для остальных - нет.

In [163]:
arrays=array([array([1.59317582, 0.43959732, 0.45791466, 1.32246291, 1.81912974,
       0.60823814, 0.84957669]),
array([0.59925339, 1.32309979, 1.81849287, 0.44503116, 1.51425836,
       0.89589361, 0.73844345]),
array([1.58401055, 1.37131369, 0.18626841, 2.52319093, 0.61840173,
       1.57698146, 1.56461119]),
array([2.40470309, 0.61696029, 1.32101377, 1.82057888, 1.05863359,
       0.8084145 , 1.39447384]),
array([0.27863145, 0.48424409, 0.99983325, 2.1417594 , 0.24891867,
       0.17828762, 1.90322754]),
array([1.61936581, 1.50528048, 1.63631218, 0.94832561, 2.19326704,
       0.3661111 , 1.15611574]),
array([0.88781097, 1.74207824, 1.39951442, 0.98285695, 2.1587357 ,
       0.98285695, 1.27092473]),
array([1.00000000e-15, 5.29340484e-01, 2.61225217e+00, 5.29340484e-01,
       1.62286426e+00, 7.45832107e-01, 1.16891836e+00]),
array([1.00000000e-15, 2.44194997e+00, 6.99642685e-01, 6.48314520e-01,
       4.24676043e-01, 1.90532250e+00, 1.23627015e+00]),
array([0.22899094, 1.80142582, 1.34016683, 1.80142582, 0.99973642,
       0.74821073, 0.28246961])])

In [173]:
s = get_simplified_matrix(arrays[9])

scipy.io.savemat('a', {'data':s}, do_compression=False)

Тут 0, 1, 2, 3, 4, 5, 6 - коположительные   
Остальные - нет.

Проверь, что они реально удовлетворяют условиям - кажись нет, Роланд ровно это проверял.

#### 3. Проверяем сам storm
Хотим проверить одну вещь:
* Есть ли в части объединения этих пространств, не лежащих ни в одном из них, коположительные матрицы

In [35]:
g = ALL_GRAPHS['storm']
I = find_stable_sets(g)
A, var_matr, num_params = build_triple_equalities(I, g.shape[0])

In [75]:
def get_graph_starts(g):
    I = find_stable_sets(g)
    A, var_matr, num_params = build_triple_equalities(I, g.shape[0])
    
    num_combs = total_num_of_combinations(I, var_matr, num_params, only_nonequivalent=True)

    chain_starts = []

    for i, (addA, addB, opening) in tqdm(enumerate(build_variance_equalities_iterator(
            I, var_matr, num_params, only_nonequivalent=True)), total=num_combs):
        
        rank, null_basis = get_rank_and_nullspace(np.array(A + addA, dtype=np.float64))
        
        if len(null_basis) != 0 and len(null_basis[0]) != 0:
            get_new_starts(chain_starts, [i, rank, null_basis, opening])

    print("{} x {}".format(len(A), len(A[0])), "{} x {}".format(len(addA), len(addA[0])))
    print(len(chain_starts), "dimensions:", ",".join([str(len(x[2][0])) for x in chain_starts]))

    print("openings:")
    for x in chain_starts: 
        pprint(x[-1])
    
    return chain_starts

In [36]:
%%time

num_combs = total_num_of_combinations(I, var_matr, num_params, only_nonequivalent=True)

chain_starts = []

for i, (addA, addB, opening) in tqdm(enumerate(build_variance_equalities_iterator(
        I, var_matr, num_params, only_nonequivalent=True)), total=num_combs):
    rank, null_basis = get_rank_and_nullspace(np.array(A + addA, dtype=np.float64))
    if len(null_basis) != 0:
        get_new_starts(chain_starts, [i, rank, null_basis, opening])

print("{} x {}".format(len(A), len(A[0])), "{} x {}".format(len(addA), len(addA[0])))
print(len(chain_starts), "dimensions:", ",".join([str(len(x[2][0])) for x in chain_starts]))

print("openings:")
for x in chain_starts: 
    pprint(x[-1])

100%|██████████| 131072/131072 [09:37<00:00, 227.08it/s]

19 x 45 35 x 45
2 dimensions: 7,7
openings:
defaultdict(<class 'list'>,
            {0: [([2, 27, 14], 1), ([8, 29, 10], -1), ([8, 38, 12], -1)],
             3: [([1, 19, 30], 1), ([1, 21, 32], 1), ([7, 37, 30], 1)],
             5: [([7, 28, 26], 1), ([8, 29, 26], 1)],
             9: [([11, 30, 19], 1), ([11, 32, 21], 1)],
             13: [([16, 29, 26], 1)],
             15: [([11, 30, 37], 1)],
             17: [([1, 6, 27], 1)],
             22: [([1, 4, 37], 1), ([18, 30, 37], 1), ([18, 31, 40], 1)],
             23: [([1, 4, 38], 1)],
             24: [([10, 14, 32], 1), ([28, 40, 31], -1)],
             25: [([2, 7, 37], 1), ([2, 8, 38], 1), ([10, 16, 38], 1)],
             34: [([11, 12, 38], 1)],
             35: [([19, 18, 31], 1), ([37, 33, 31], 1)],
             36: [([4, 1, 21], 1), ([12, 11, 32], -1), ([19, 18, 32], -1)],
             39: [([20, 18, 32], 1)],
             42: [([6, 2, 28], 1)],
             43: [([6, 2, 29], 1), ([14, 10, 29], 1)],
             44: [([

In [37]:
first_space  = chain_starts[0][2]
second_space = chain_starts[1][2]

#### Проверяем центральную точку

In [38]:
center = get_center_point(g, var_matr)
get_matrix_at_point(var_matr, center)

array([[ 1. ,  1. , -0.5, -0.5,  1. , -0.5,  1. , -0.5, -0.5, -0.5],
       [ 1. ,  1. ,  1. , -0.5, -0.5, -0.5,  1. , -0.5,  1. , -0.5],
       [-0.5,  1. ,  1. ,  1. , -0.5, -0.5, -0.5, -0.5,  1. ,  1. ],
       [-0.5, -0.5,  1. ,  1. ,  1. ,  1. , -0.5, -0.5, -0.5, -0.5],
       [ 1. , -0.5, -0.5,  1. ,  1. , -0.5, -0.5, -0.5, -0.5,  1. ],
       [-0.5, -0.5, -0.5,  1. , -0.5,  1. ,  1. ,  1. , -0.5, -0.5],
       [ 1. ,  1. , -0.5, -0.5, -0.5,  1. ,  1. ,  1. , -0.5, -0.5],
       [-0.5, -0.5, -0.5, -0.5, -0.5,  1. ,  1. ,  1. ,  1. ,  1. ],
       [-0.5,  1. ,  1. , -0.5, -0.5, -0.5, -0.5,  1. ,  1. ,  1. ],
       [-0.5, -0.5,  1. , -0.5,  1. , -0.5, -0.5,  1. ,  1. ,  1. ]])

In [39]:
check_lem6(var_matr, I, center)

-2.220446049250313e-16

Код говорит, что она копложительна

#### Проверяем по отдельности

In [53]:
np.savetxt("second_base.txt", second_space)

In [59]:
direction = get_random_direction(first_space)

In [60]:
point = center + 0.1 * direction
s=get_matrix_at_point(var_matr, point)

In [61]:
check_lem6(var_matr, I, point)

-3.3306690738754696e-16

In [62]:
s

array([[ 1.        ,  0.99999455, -0.43804646, -0.49416199,  0.99991935,
        -0.50702983,  0.99936852, -0.50987196, -0.49890566, -0.42758519],
       [ 0.99999455,  1.        ,  0.99872006, -0.49702876, -0.49403572,
        -0.50987196,  0.99948036, -0.50702983,  0.99983237, -0.42459918],
       [-0.43804646,  0.99872006,  1.        ,  0.99798796, -0.44942783,
        -0.55273003, -0.52754393, -0.54997641,  0.99762656,  0.99993267],
       [-0.49416199, -0.49702876,  0.99798796,  1.        ,  0.99999406,
         0.99988952, -0.5247408 , -0.49591633, -0.50690464, -0.57459231],
       [ 0.99991935, -0.49403572, -0.44942783,  0.99999406,  1.        ,
        -0.49604244, -0.52180436, -0.49890566, -0.50987196,  0.99694345],
       [-0.50702983, -0.50987196, -0.55273003,  0.99988952, -0.49604244,
         1.        ,  0.99955214,  0.99999455, -0.49403572, -0.56236343],
       [ 0.99936852,  0.99948036, -0.52754393, -0.5247408 , -0.52180436,
         0.99955214,  1.        ,  0.99964547

In [63]:
scipy.io.savemat('a', {'data':s}, do_compression=False)

Проверил три случайных направления - для них норм

In [968]:
direction = get_random_direction(second_space)
point = center + 0.1 * direction
s=get_matrix_at_point(var_matr, point)

In [969]:
check_lem6(var_matr, I, point)

-4.440892098500626e-16

In [478]:
s

array([[ 1.        ,  0.83822058, -0.3419432 , -0.2308309 ,  0.93647758,
        -0.60402616,  0.99383016, -0.94091686, -0.11229967, -0.75876868],
       [ 0.83822058,  1.        ,  0.7948362 , -0.72409126, -0.05524488,
        -0.94091686,  0.89353303, -0.60402616,  0.91486948, -0.28080841],
       [-0.3419432 ,  0.7948362 ,  1.        ,  0.99327347, -0.64980795,
        -0.54238132,  0.23289275,  0.00351535,  0.97217642,  0.87155239],
       [-0.2308309 , -0.72409126,  0.99327347,  1.        ,  0.72865322,
         0.91486948, -0.33732403, -0.11229967, -0.94091686, -0.45862222],
       [ 0.93647758, -0.05524488, -0.64980795,  0.72865322,  1.        ,
        -0.28613965, -0.89053313, -0.76237804, -0.45367499,  0.97381136],
       [-0.60402616, -0.94091686, -0.54238132,  0.91486948, -0.28613965,
         1.        ,  0.69071415,  0.83822058, -0.72409126, -0.0607948 ],
       [ 0.99383016,  0.89353303,  0.23289275, -0.33732403, -0.89053313,
         0.69071415,  1.        ,  0.97331512

In [475]:
scipy.io.savemat('a', {'data':s}, do_compression=False)

Проверил три случайных направлений - норм. (Для каких-то направлений нужно было взять шаг 0.1, но всё равно матрица была в многообразии)

#### Проверяем объединение

In [509]:
_, v, d = np.linalg.svd(np.hstack([first_space, second_space]).T)

In [511]:
tol = v.max() * max(np.hstack([first_space, second_space]).shape) * np.finfo(first_space.dtype).eps
nnz = (v >= tol).sum()

In [514]:
concat_space = d[:nnz].T

In [519]:
concat_space.shape

(45, 8)

In [868]:
# get direction that doesn't belong to base subspaces

direction = get_random_direction(concat_space)

In [878]:
print(a_is_subspace_of_b(np.array([direction]).T, first_space))
print(a_is_subspace_of_b(np.array([direction]).T, second_space))

False
False


In [882]:
point = center + 1e-1 * direction
s=get_matrix_at_point(var_matr, point)

In [883]:
s

array([[ 1.        ,  0.99996276, -0.49694337, -0.50281231,  0.99999707,
        -0.50374154,  0.99998673, -0.49626783, -0.49636562, -0.49172855],
       [ 0.99996276,  1.        ,  0.99999386, -0.49533397, -0.49788464,
        -0.49626783,  0.99999395, -0.50374154,  0.99999131, -0.49922467],
       [-0.49694337,  0.99999386,  1.        ,  0.99997704, -0.49484247,
        -0.49930715, -0.50025489, -0.50676568,  0.99999978,  0.99998201],
       [-0.50281231, -0.49533397,  0.99997704,  1.        ,  0.99999568,
         0.99999942, -0.49835267, -0.50091231, -0.50081482, -0.50542406],
       [ 0.99999707, -0.49788464, -0.49484247,  0.99999568,  1.        ,
        -0.50583   , -0.50488579, -0.49836676, -0.49426393,  0.99999881],
       [-0.50374154, -0.49626783, -0.49930715,  0.99999942, -0.50583   ,
         1.        ,  0.9999994 ,  0.99996276, -0.49988377, -0.50449589],
       [ 0.99998673,  0.99999395, -0.50025489, -0.49835267, -0.50488579,
         0.9999994 ,  1.        ,  0.99997161

In [884]:
check_lem6(var_matr, I, point)

-3.3306690738754696e-16

In [873]:
scipy.io.savemat('a', {'data':s}, do_compression=False)

In [875]:
copos_directions = [
   [ -0.07816216619234312 , -0.16672522621276023 , -0.11347128367631763 , 0.0011252746761362662 ,0.09671757697226963 ,-0.09169722056166929 ,0.17487974316461422 ,-0.15053842062387043 ,-0.12811790954570404 ,-0.10487209392412418 ,-0.035309117483975416 ,-0.27272259481302136 ,0.174879743164613 ,-0.1698593867540129 ,0.0967175769722704 ,-0.1210588995130135 ,-0.20628007573804644 ,-0.0532539425364418 ,-0.16785050088889705 ,0.0700076492404902 ,-0.042654394473312505 ,-0.008154516951853938 ,-0.016186805588889536 ,-0.03860731666705553 ,0.23741347732904702 ,-0.21018886064858983 ,-0.20516850423798655 ,-0.06140845948829607 ,0.26400970430018783 ,0.24158919322202146 ,0.09784285164840724 ,0.21050489536220945 ,0.17600501784075212 ,-0.1516636953000079 ,-0.0664425190749736 ,0.11266204371380223 ,-0.07816216619234445 ,0.05382084365159984 ,0.03140033257343362 ,-0.03449987752145654 ,-0.058841200062202316 ,-0.03642068898403502 ,0.3254181637884847 ,0.302997652710317 ,-0.02242051107816685],
[-0.27470036720852387 ,
0.11223330193181104 ,
-0.10346446789065462 ,
-0.07700922637242001 ,
-0.13769346568101634 ,
0.16397934714636758 ,
0.13700690152750777 ,
0.13342157338219765 ,
0.3031903248838833 ,
-0.11154673777830303 ,
0.17123589931786906 ,
0.07769579052592844 ,
0.1370069015275076 ,
-0.11072102006215567 ,
-0.1376934656810157 ,
-0.13273500922868955 ,
0.02848995767535958 ,
0.2156977698224656 ,
0.1892425283042312 ,
0.025460163749204793 ,
-0.009369502313783397 ,
-0.24924020345931897 ,
-0.021188271450386723 ,
-0.1909570229520721 ,
0.09354010879194083 ,
0.03422899779036152 ,
0.06051487925571332 ,
-0.03354243363685358 ,
-0.02995710549154304 ,
-0.19972585699322853 ,
-0.21470269205343634 ,
-0.17987302599044777 ,
0.059997675155087644 ,
0.2104307997546181 ,
0.049205832850568704 ,
0.03482966606298851 ,
-0.27470036720852375 ,
0.004271892298818064 ,
-0.1654968592028671 ,
0.2398707011455354 ,
-0.030557773764170192 ,
0.13921097773751545 ,
0.003585328145309792 ,
-0.1661834233563754 ,
-0.16976875150168572 ,]
] # дают коположительные матрицы даже при шаге 0.1
# остальные дают коположитльные матрицы только при шаге 1e-7, что в матлабе эквивалентно центральное точке 
# по лемме 6 обычно всё ясно - если она выполнена, то это скорее всего коположительная матрица
# надо на них подробнее посмотреть

# дающие коположительные матрицы лучи могут быть из другого страта?

In [877]:
print(a_is_subspace_of_b(np.array([np.array(copos_directions[0])]).T, first_space))
print(a_is_subspace_of_b(np.array([np.array(copos_directions[0])]).T, second_space))
print(a_is_subspace_of_b(np.array([np.array(copos_directions[0])]).T, concat_space))

False
False
True


### Оставшиеся графы на 10 вершинах из статьи

In [101]:
rest_of_the_graphs = {
    "5": np.array([
        [1, 1, 1, 1, 0, 1, 0, 0, 0, 0],
        [1, 1, 0, 1, 0, 0, 0, 0, 1, 1],
        [1, 0, 1, 0, 1, 0, 0, 1, 0, 1],
        [1, 1, 0, 1, 1, 0, 0, 0, 1, 0],
        [0, 0, 1, 1, 1, 0, 1, 0, 1, 0],
        [1, 0, 0, 0, 0, 1, 1, 1, 0, 0],
        [0, 0, 0, 0, 1, 1, 1, 0, 0, 1],
        [0, 0, 1, 0, 0, 1, 0, 1, 1, 1],
        [0, 1, 0, 1, 1, 0, 0, 1, 1, 0],
        [0, 1, 1, 0, 0, 0, 1, 1, 0, 1]
    ]),
    "6": np.array([
        [1, 1, 1, 0, 1, 0, 0, 0, 1, 0],
        [1, 1, 1, 0, 0, 1, 1, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 1],
        [0, 0, 1, 1, 1, 0, 1, 1, 0, 0],
        [1, 0, 0, 1, 1, 1, 0, 0, 1, 0],
        [0, 1, 0, 0, 1, 1, 1, 0, 0, 1],
        [0, 1, 0, 1, 0, 1, 1, 1, 0, 0],
        [0, 0, 0, 1, 0, 0, 1, 1, 1, 1],
        [1, 0, 0, 0, 1, 0, 0, 1, 1, 1],
        [0, 0, 1, 0, 0, 1, 0, 1, 1, 1]
    ]),
    "7": np.array([
        [1, 1, 0, 0, 0, 0, 1, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 1, 0, 0],
        [0, 1, 1, 1, 0, 0, 0, 1, 1, 1],
        [0, 0, 1, 1, 1, 0, 0, 0, 1, 0],
        [0, 0, 0, 1, 1, 1, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 1, 1, 0, 1],
        [1, 0, 0, 0, 0, 1, 1, 0, 1, 1],
        [0, 1, 1, 0, 0, 1, 0, 1, 1, 1],
        [0, 0, 1, 1, 0, 0, 1, 1, 1, 1],
        [0, 0, 1, 0, 0, 1, 1, 1, 1, 1]
    ]),
    "8": np.array([
        [1, 1, 0, 0, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 1, 0, 0, 0],
        [0, 1, 1, 1, 0, 0, 0, 1, 1, 0],
        [0, 0, 1, 1, 1, 0, 0, 0, 1, 1],
        [1, 0, 0, 1, 1, 1, 0, 0, 0, 1],
        [1, 0, 0, 0, 1, 1, 0, 1, 0, 0],
        [0, 1, 0, 0, 0, 0, 1, 1, 0, 1],
        [0, 0, 1, 0, 0, 1, 1, 1, 1, 0],
        [0, 0, 1, 1, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 1, 1, 0, 1, 0, 1, 1]
    ]),
    "9": np.array([
        [1, 1, 0, 0, 1, 0, 1, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 1, 0, 0, 0],
        [0, 1, 1, 1, 0, 1, 0, 0, 0, 1],
        [0, 0, 1, 1, 1, 0, 0, 0, 1, 1],
        [1, 0, 0, 1, 1, 1, 0, 0, 1, 0],
        [0, 0, 1, 0, 1, 1, 0, 1, 0, 0],
        [1, 1, 0, 0, 0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
        [0, 0, 0, 1, 1, 0, 0, 1, 1, 1],
        [0, 0, 1, 1, 0, 0, 0, 1, 1, 1]
    ]),
    "10": np.array([
        [1, 1, 0, 0, 1, 0, 1, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 0, 0, 0, 1, 0, 0],
        [0, 0, 1, 1, 1, 0, 0, 1, 0, 1],
        [1, 0, 0, 1, 1, 1, 0, 0, 0, 1],
        [0, 0, 0, 0, 1, 1, 1, 0, 1, 1],
        [1, 0, 0, 0, 0, 1, 1, 1, 1, 0],
        [0, 0, 1, 1, 0, 0, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
        [0, 0, 0, 1, 1, 1, 0, 0, 1, 1]
    ]),
    "11": np.array([
        [1, 1, 0, 0, 0, 1, 0, 1, 1, 0],
        [1, 1, 1, 0, 0, 0, 0, 0, 1, 0],
        [0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 0, 0, 0, 0, 1],
        [0, 0, 0, 1, 1, 1, 1, 0, 0, 1],
        [1, 0, 0, 0, 1, 1, 1, 1, 0, 0],
        [0, 0, 0, 0, 1, 1, 1, 1, 0, 1],
        [1, 0, 0, 0, 0, 1, 1, 1, 1, 0],
        [1, 1, 0, 0, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 1, 1, 0, 1, 0, 1, 1]
    ]),
    "12": np.array([
        [1, 1, 0, 0, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 1, 1, 0],
        [0, 1, 1, 1, 0, 0, 0, 0, 1, 1],
        [0, 0, 1, 1, 1, 0, 0, 0, 1, 1],
        [1, 0, 0, 1, 1, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 1, 0, 1],
        [0, 1, 0, 0, 0, 0, 1, 1, 1, 0],
        [0, 1, 1, 1, 0, 0, 0, 1, 1, 0],
        [0, 0, 1, 1, 0, 0, 1, 0, 0, 1]
    ]),
    "13": np.array([
        [1, 1, 0, 0, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 1, 1, 0],
        [0, 1, 1, 1, 0, 0, 0, 1, 1, 0],
        [0, 0, 1, 1, 1, 0, 0, 0, 1, 1],
        [1, 0, 0, 1, 1, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 0, 1, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 1, 1, 1, 0, 1],
        [0, 1, 1, 0, 0, 0, 1, 1, 0, 0],
        [0, 1, 1, 1, 0, 1, 0, 0, 1, 1],
        [0, 0, 0, 1, 1, 0, 1, 0, 1, 1]
    ]),
    "14": np.array([
        [1, 1, 0, 0, 1, 1, 0, 1, 0, 0],
        [1, 1, 1, 0, 0, 0, 1, 0, 0, 0],
        [0, 1, 1, 1, 0, 0, 0, 1, 1, 0],
        [0, 0, 1, 1, 1, 0, 0, 0, 1, 0],
        [1, 0, 0, 1, 1, 1, 0, 0, 0, 0],
        [1, 0, 0, 0, 1, 1, 0, 1, 0, 1],
        [0, 1, 0, 0, 0, 0, 1, 0, 0, 1],
        [1, 0, 1, 0, 0, 1, 0, 1, 1, 1],
        [0, 0, 1, 1, 0, 0, 0, 1, 1, 1],
        [0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
    ]),
    "15": np.array([
        [1, 1, 0, 0, 1, 1, 0, 0, 0, 0],
        [1, 1, 1, 0, 0, 0, 0, 1, 0, 0],
        [0, 1, 1, 1, 0, 0, 0, 0, 1, 0],
        [0, 0, 1, 1, 1, 0, 0, 0, 1, 1],
        [1, 0, 0, 1, 1, 1, 0, 0, 0, 1],
        [1, 0, 0, 0, 1, 1, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
        [0, 1, 0, 0, 0, 0, 1, 1, 0, 1],
        [0, 0, 1, 1, 0, 1, 1, 0, 1, 0],
        [0, 0, 0, 1, 1, 0, 1, 1, 0, 1]
    ]),
    "16": np.array([
        [1, 1, 1, 1, 0, 0, 0, 1, 0, 0],
        [1, 1, 1, 0, 0, 0, 1, 0, 0, 1],
        [1, 1, 1, 1, 1, 0, 0, 0, 0, 0],
        [1, 0, 1, 1, 1, 1, 0, 0, 0, 0],
        [0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
        [0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
        [0, 1, 0, 0, 1, 1, 1, 1, 0, 0],
        [1, 0, 0, 0, 0, 1, 1, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
        [0, 1, 0, 0, 0, 0, 0, 0, 1, 1]
    ])
}

In [102]:
for x in rest_of_the_graphs.values():
    if not is_copirr(x):
        print(x)

In [103]:
del rest_of_the_graphs["5"]
del rest_of_the_graphs["6"]
del rest_of_the_graphs["7"]
del rest_of_the_graphs["8"]
del rest_of_the_graphs["9"]
del rest_of_the_graphs["10"]
del rest_of_the_graphs["11"]
#del rest_of_the_graphs["12"] # this takes 10 days...
del rest_of_the_graphs["13"] 
del rest_of_the_graphs["14"]
del rest_of_the_graphs["15"] # way too much base openings...
del rest_of_the_graphs["16"] 

In [104]:
res = {}

for name, g in rest_of_the_graphs.items():
    
    print("Graph {}".format(name))
    res[name] = get_graph_starts(g)

  0%|          | 13/134217728 [00:00<325:14:20, 114.63it/s]

Graph 12


100%|██████████| 134217728/134217728 [127:24:18<00:00, 292.63it/s]        


21 x 45 44 x 45
1 dimensions: 7
openings:
defaultdict(<class 'list'>,
            {0: [([2, 26, 13], 1)],
             3: [([1, 20, 31], 1),
                 ([1, 21, 32], -1),
                 ([7, 40, 31], -1),
                 ([8, 43, 32], -1),
                 ([7, 44, 34], -1)],
             4: [([1, 21, 36], 1),
                 ([2, 27, 36], 1),
                 ([8, 43, 36], 1),
                 ([7, 44, 38], -1)],
             9: [([11, 31, 20], 1)],
             14: [([10, 25, 36], 1), ([16, 34, 32], 1), ([16, 38, 36], 1)],
             15: [([11, 31, 40], 1), ([16, 34, 33], -1), ([16, 38, 37], -1)],
             17: [([1, 5, 26], 1), ([1, 6, 27], 1), ([19, 36, 27], 1)],
             22: [([1, 5, 40], 1), ([18, 31, 40], 1)],
             23: [([1, 6, 43], 1), ([18, 32, 43], 1), ([19, 36, 43], 1)],
             24: [([10, 13, 31], 1)],
             28: [([2, 5, 40], 1)],
             29: [([2, 6, 43], 1), ([10, 12, 38], -1), ([25, 36, 43], 1)],
             30: [([11, 16, 38]